# 混合检索的因果逻辑分析

## 一、问题根源：为什么需要混合检索？

### 1.1 向量检索的本质缺陷

向量检索的核心假设是：**语义相近的文本在向量空间中距离更近**。这个假设大体正确，但存在根本性的局限。

**根本原因**：Embedding 模型是对语义的"有损压缩"。

一段文本可能包含无限丰富的语义信息，但 Embedding 模型必须将其压缩到固定维度（如 768 维或 1536 维）的向量中。这个压缩过程必然导致信息损失，就像把一张高清照片压缩成缩略图——大体轮廓还在，但细节丢失了。

**具体表现为以下几类问题**：

| 缺陷类型 | 具体表现 | 因果解释 |
|----------|----------|----------|
| 否定词失效 | "我喜欢这个产品"和"我不喜欢这个产品"的向量相似度很高 | 两个句子共享绝大部分相同的词汇，模型训练时倾向于捕捉共性，导致一个"不"字的语义差异被淹没 |
| 专业术语理解差 | 医学、法律等领域的检索效果明显下降 | Embedding 模型的训练语料以通用文本为主，专业领域数据稀疏，模型未能学到准确的领域语义表示 |
| 精确匹配弱 | 搜索订单号"ORD-2024-001"时，可能返回其他订单 | 向量编码的是"模糊语义"，天然不擅长处理需要精确匹配的场景（如 ID、代码、专有名词） |
| 长尾知识缺失 | 新词、罕见概念无法正确表示 | 训练数据中未出现或出现极少的概念，模型无法学到有意义的表示 |

**量化表现**：在标准评测基准（如 BEIR）上，向量检索在通用领域的召回率可达 85-95%，但在专业领域可能降至 60-80%。这意味着每 10 个相关文档中，可能有 2-4 个被漏掉。

### 1.2 关键词检索（BM25）的本质缺陷

传统的关键词检索（以 BM25 为代表）走向了另一个极端：**只看词汇是否重叠，完全不理解语义**。

**根本原因**：BM25 基于词频统计，认为查询词在文档中出现次数越多、在整个语料库中越稀有，文档就越相关。这是一种纯粹的"符号匹配"，不涉及任何语义理解。

**具体问题**：

| 缺陷类型 | 具体表现 | 因果解释 |
|----------|----------|----------|
| 同义词失效 | "如何购买苹果手机"匹配不到"iPhone 购买指南" | "苹果手机"和"iPhone"没有词汇重叠，BM25 认为完全不相关 |
| 表达多样性 | "怎么退货"匹配不到"退款流程说明" | 用户的口语表达与文档的正式表达词汇不同 |
| 概念理解缺失 | "经济实惠的笔记本"匹配不到"性价比高的电脑" | 需要理解"经济实惠≈性价比高"、"笔记本⊂电脑"，BM25 无法做到 |

### 1.3 互补关系的形成

将两种方法的优缺点并列，会发现一个有趣的现象：

```
向量检索的优势 = BM25 的劣势
向量检索的劣势 = BM25 的优势
```

具体对比：

| 能力维度 | 向量检索 | BM25 检索 |
|----------|----------|-----------|
| 语义理解 | ✅ 强 | ❌ 无 |
| 同义词处理 | ✅ 自动支持 | ❌ 需要人工同义词表 |
| 精确匹配 | ❌ 弱 | ✅ 强 |
| 专业术语 | ❌ 依赖训练数据 | ✅ 只要词汇匹配就行 |
| 否定词处理 | ❌ 差 | ✅ 能区分有无否定词 |

**关键洞察**：两种方法的"失败模式"不同。向量检索漏掉的文档，BM25 可能找得到；BM25 漏掉的文档，向量检索可能找得到。这就是混合检索有效的根本原因——**用多个"有缺陷但互补"的方法，逼近一个"更完美"的效果**。

---

## 二、Milvus 的处理方法

Milvus 作为专业的向量数据库，通过**多向量搜索（Hybrid Search）**能力来应对单一向量检索的局限性。核心思想是：在一次查询中同时搜索多个向量字段，然后将结果融合。

### 2.1 理解向量数据库的职责边界

在深入 Milvus 的具体能力之前，需要先明确一个关键认知：

**向量数据库只处理向量，不理解语义。**

对于 Milvus 来说，不管存储的是文本向量、图像向量还是音频向量，它看到的都只是一串浮点数。向量代表什么含义、能否跨模态比较、语义理解是否准确——这些完全取决于生成向量的 Embedding 模型，与向量数据库无关。

| 组件 | 职责 |
|------|------|
| Embedding 模型 | 把各种数据变成向量，决定语义质量和跨模态能力 |
| 向量数据库 | 存储向量，快速检索相似向量，融合多路结果 |

这意味着：Milvus 的多向量能力是提供了"基础设施"，但效果好不好，取决于你选择的 Embedding 模型。

### 2.2 三种主要的向量类型

Milvus 支持存储和检索多种类型的向量，每种向量有不同的特点和适用场景：

**密集向量（Dense Vector）**

通过神经网络 Embedding 模型生成。特点是固定维度（如 768 或 1536 维），每个维度都有值。典型模型包括 OpenAI text-embedding、BERT、BGE 等。

擅长语义理解和同义词匹配。例如"如何捕获异常"和"怎么处理错误"虽然词汇不同，但密集向量会比较接近。

弱点是精确关键词匹配较差，对否定词和专业术语的处理也不够好。

**稀疏向量（Sparse Vector）**

通过词频统计或稀疏编码模型生成。特点是维度数等于词表大小（可能几万维），绝大部分维度为零，只有文本中出现的词对应的维度有非零值。典型方法包括 BM25、TF-IDF、SPLADE、BGE-M3 等。

本质上是把传统的关键词匹配（如 BM25）表示成向量形式。擅长精确关键词匹配和专业术语检索。

弱点是不理解语义，无法处理同义词。"异常处理"和"错误捕获"在稀疏向量看来完全不相关。

**多模态向量（Multi-modal Vector）**

通过多模态对齐模型生成。典型模型包括 CLIP、BLIP、Chinese-CLIP 等。

关键特点是：可以把不同模态的数据（文本、图像、音频等）映射到同一个向量空间。这意味着可以用文本查询图像，或用图像查询文本。

实现跨模态搜索的前提是：入库时用 CLIP 的图像编码器生成图像向量，查询时用 CLIP 的文本编码器生成查询向量。因为两个编码器输出的向量在同一空间，所以可以计算相似度。如果使用普通的文本模型和图像模型，生成的向量不在同一空间，无法互相搜索。

### 2.3 多向量搜索的工作原理

Milvus 允许一个 Collection 中定义多个向量字段。例如一个电商商品表可以同时包含：

- dense_vector：商品描述的密集向量
- sparse_vector：商品描述的稀疏向量
- image_vector：商品图片的 CLIP 向量

**入库时**，对同一条商品数据，分别用不同的模型生成不同类型的向量，存入对应的字段。

**查询时**，用户输入一个文本查询，系统需要：

1.  用密集向量模型生成查询的密集向量，搜索 dense_vector 字段，得到结果集 A
2. 用稀疏向量模型生成查询的稀疏向量，搜索 sparse_vector 字段，得到结果集 B
3. 用 CLIP 文本编码器生成查询的图像空间向量，搜索 image_vector 字段，得到结果集 C
4.  将三个结果集融合，得到最终排序

**融合策略**方面，Milvus 内置了两种方法：

RRF（倒数排名融合）：不看具体分数，只看每条结果在各路中的排名。排名越靠前，得分越高。多路都靠前的结果，融合后排名更高。这种方法简单稳定，避免了不同向量类型分数量纲不同的问题。

WeightedRanker（加权融合）：对各路分数归一化后加权求和。可以手动调整各路的权重，适合对业务场景有明确理解的情况。

### 2.4 多向量搜索有效的原因

回到我们在第一节讨论的核心问题：不同类型的向量有不同的"失败模式"。

密集向量会漏掉需要精确匹配的查询，稀疏向量会漏掉需要语义理解的查询，两者的漏选集合基本不重叠。多向量搜索的价值在于：同时执行多种搜索，取并集，用互补性覆盖彼此的盲区。

举例来说，用户搜索"Python try except 教程"：

- 密集向量可能匹配到"Python 异常处理指南"（语义相近），但漏掉标题就叫"try-except 详解"的文章（精确匹配弱）
- 稀疏向量可能匹配到"try-except 详解"（关键词匹配），但漏掉"异常处理指南"（不理解同义词）
- 两者融合后，两篇文章都能被检索到

### 2.5 Milvus 方案的局限性

需要清醒认识到，Milvus 的多向量能力只是在"检索层面"缓解问题，并没有从根本上解决 Embedding 模型的语义理解缺陷。

Milvus 能做的：
- 高效存储和检索各种类型的向量
- 支持一次查询搜索多个向量字段
- 提供内置的结果融合策略

Milvus 不能解决的：
- Embedding 模型本身对否定词、专业术语的理解缺陷
- 如果模型把语义理解错了，Milvus 只能"快速地返回错误的结果"
- 需要复杂语义推理的场景（如多跳问答）

因此，多向量搜索是必要的，但不是充分的。后续的 Rerank 精排环节仍然不可或缺。

---

## 三、最佳实践流程：分层处理的因果逻辑

基于对向量检索局限性的认识，业界逐渐形成了一套分层处理的最佳实践。这不是某个人拍脑袋想出来的，而是在实践中不断试错后收敛出的方案。以下分析每一层存在的原因和相互之间的逻辑关系。

### 3.1 完整流程概览

```
用户查询
   │
   ▼
┌─────────────────────────────────┐
│  第一层：查询理解与改写          │
└─────────────────────────────────┘
   │
   ▼
┌─────────────────────────────────┐
│  第二层：多路并行召回            │
│   ├── 密集向量召回              │
│   ├── 稀疏向量 / BM25 召回      │
│   └── （可选）规则/关键词召回   │
└─────────────────────────────────┘
   │
   ▼
┌─────────────────────────────────┐
│  第三层：融合与去重              │
└─────────────────────────────────┘
   │
   ▼
┌─────────────────────────────────┐
│  第四层：Rerank 精排            │
└─────────────────────────────────┘
   │
   ▼
┌─────────────────────────────────┐
│  第五层：LLM 生成回答            │
└─────────────────────────────────┘
```

### 3.2 第一层：查询理解与改写

**要解决的问题**：用户输入的查询往往不是"检索友好"的。

用户输入的典型问题：
- **过于口语化**："这玩意儿咋用啊"
- **过于简短**："退货"（缺少上下文）
- **过于冗长**：包含大量背景描述，核心问题被淹没
- **包含歧义**："苹果怎么样"（水果？公司？手机？）
- **拼写错误**："pytohn 怎么学"

如果直接拿用户的原始输入去检索，效果往往不佳。

**查询改写的作用**：

| 改写类型 | 示例 | 目的 |
|----------|------|------|
| 规范化 | "这玩意儿咋用" → "产品使用方法" | 转换为书面表达，匹配文档风格 |
| 扩展 | "退货" → "退货流程、退货政策、如何申请退货" | 补充可能的检索角度 |
| 精简 | 长段落 → 提取核心问题 | 去除噪音，聚焦关键信息 |
| 消歧 | 结合上下文确定"苹果"指的是什么 | 避免检索到不相关的内容 |
| 纠错 | "pytohn" → "python" | 修复拼写错误 |

**因果链条**：

```
用户表达习惯 ≠ 文档写作风格
  → 直接用用户输入检索，匹配度低
    → 需要把用户意图"翻译"成检索友好的形式
      → 用 LLM 理解用户意图并改写查询
        → 改写后的查询更容易匹配到相关文档
```

### 3. 3 第二层：多路并行召回

**要解决的问题**：单一召回方法有盲区，会漏掉相关文档。

**为什么叫"召回"而不是直接"检索"**：

这里需要理解一个关键概念——**召回和精排的分工**。

在一个典型的 RAG 系统中，知识库可能有几十万甚至上百万篇文档。对每一篇文档都做精细的相关性判断，计算成本太高，不现实。因此需要分两步：

1.  **召回（Recall）**：用低成本的方法，快速从海量文档中筛出一个较小的候选集（如 Top 50）。要求：速度快，宁可多选不能漏选。
2. **精排（Ranking）**：用高成本但高精度的方法，对候选集中的文档精细排序。要求：排序准确。

召回阶段追求的是**高召回率**（别漏掉好的），可以容忍一定的误召回（选进来一些不那么好的）；精排阶段会把误召回的剔除。

**为什么要多路召回**：

既然向量召回和 BM25 召回各有盲区，那就**都做**，然后取并集。

```
假设：
- 向量召回的候选集：{A, B, C, D, E}
- BM25 召回的候选集：{A, C, F, G, H}

两者有重叠（A, C），但也有各自独有的（向量独有 B, D, E；BM25 独有 F, G, H）

合并后的候选集：{A, B, C, D, E, F, G, H}
  → 候选更全面
  → 向量漏掉的 F, G, H 被 BM25 补上了
  → BM25 漏掉的 B, D, E 被向量补上了
```

**因果链条**：

```
单一方法有盲区
  → 不同方法的盲区不同
    → 多种方法并行执行
      → 取并集作为候选
        → 大大降低漏选概率
          → 召回率提升
```

### 3. 4 第三层：融合与去重

**要解决的问题**：多路召回的结果如何合并成一个统一的排序列表？

**具体困难**：

不同召回方法返回的分数量纲不同：
- 向量检索返回的是余弦相似度，范围是 [-1, 1]，通常是 [0. 5, 0.95] 之间
- BM25 返回的是 TF-IDF 加权分数，理论上没有上界，可能是 [5, 50] 之间

直接比较或相加没有意义。说文档 A 的向量分数是 0.85、BM25 分数是 12.3，哪个更相关？无法判断。

**解决方案：RRF（Reciprocal Rank Fusion，倒数排名融合）**

RRF 的核心思想是：**不看分数，只看排名**。

计算公式：
```
RRF_score = Σ 1/(k + rank)
```

其中 `k` 是一个常数（通常取 60），`rank` 是文档在某一路召回结果中的排名。

**RRF 的计算示例**：

假设文档 X 在向量召回中排第 2，在 BM25 召回中排第 5：
```
RRF_score(X) = 1/(60+2) + 1/(60+5) 
             = 1/62 + 1/65 
             = 0.0161 + 0. 0154 
             = 0.0315
```

假设文档 Y 在向量召回中排第 10，在 BM25 召回中排第 1：
```
RRF_score(Y) = 1/(60+10) + 1/(60+1) 
             = 1/70 + 1/61 
             = 0.0143 + 0. 0164 
             = 0.0307
```

X 的 RRF 分数更高，排在 Y 前面。

**RRF 有效的原因**：

1. **排名靠前的加分更多**：1/(60+1)=0.0164 远大于 1/(60+50)=0.0091
2. **多路都靠前的加分更多**：在两路都排前 5，比在一路排第 1、另一路排第 50 分数高
3. **不受原始分数量纲影响**：只用排名，避免了不同量纲的问题

**因果链条**：

```
不同召回方法的分数量纲不同
  → 无法直接比较或相加
    → 转换为排名（排名是可比的）
      → 用 RRF 公式融合排名
        → 得到统一的融合分数
          → 按融合分数重新排序
            → 多路结果被统一为一个列表
```

### 3.5 第四层：Rerank 精排

**要解决的问题**：召回阶段的排序不够精准，需要更精细的相关性判断。

**召回方法的局限性**：

无论是向量检索还是 BM25，都是"独立打分"模式：
- 分别计算查询和每个文档的匹配度
- 不考虑文档之间的比较
- 为了速度，模型相对简单

这种方式速度快，但精度有限。

**Reranker 的不同之处**：

Reranker（通常是一个 Cross-Encoder 模型）采用"交叉编码"模式：
- 把查询和文档**拼接在一起**作为输入
- 用一个更大的模型（如 BERT）理解拼接后的文本
- 输出一个相关性分数

**为什么交叉编码更准确**：

```
独立编码（召回阶段）：
  Query → Encoder → Query向量
  Doc   → Encoder → Doc向量
  相似度 = 向量点积
  
  问题：Query和Doc是分别编码的，无法捕捉细粒度的交互

交叉编码（Rerank阶段）：
  [Query + Doc] → Encoder → 相关性分数
  
  优势：模型可以看到Query和Doc的每一个词如何对应、交互
```

举例说明：

```
Query: "苹果手机的电池寿命"
Doc:   "iPhone 14 的续航时间可达 20 小时..."

独立编码时：
- "苹果手机" 和 "iPhone" 是分别编码的，模型需要"猜"它们是一个意思
- "电池寿命" 和 "续航时间" 是分别编码的，模型需要"猜"它们是一个意思

交叉编码时：
- 模型同时看到"苹果手机"和"iPhone"，可以学到它们在这个上下文中是同义词
- 模型同时看到"电池寿命"和"续航时间"，可以学到它们在这个上下文中是同义词
```

**为什么不直接用 Reranker 做召回**：

因为计算成本太高。

```
假设知识库有 100 万篇文档：

Reranker 直接检索：
  → 需要计算 100万次 "[Query + Doc] → 分数"
  → 每次都是一个完整的 BERT 前向传播
  → 时间：可能需要几分钟甚至更长

召回 + Rerank：
  → 召回阶段：向量检索，毫秒级返回 Top 50
  → Rerank阶段：只需要计算 50 次 BERT 前向传播
  → 时间：几百毫秒
```

**因果链条**：

```
需要高精度的相关性判断
  → Reranker（交叉编码）可以做到
    → 但 Reranker 计算成本高，无法处理全量文档
      → 先用低成本方法召回少量候选
        → 再用 Reranker 精排候选
          → 兼顾效率和精度
```

### 3.6 第五层：LLM 生成

经过前面四层，我们已经得到了与查询最相关的 Top K 篇文档（通常 K=3~5）。最后一步是把这些文档和用户的问题一起交给 LLM，生成最终的回答。

这一层本身是 RAG 的核心目标，前面四层都是为了给这一层提供高质量的上下文。

### 3.7 为什么这个顺序不能变？

这个分层顺序是有内在逻辑的，不能随意调整：

| 错误顺序 | 问题 |
|----------|------|
| 先精排再召回 | Reranker 无法处理百万级文档，计算量爆炸 |
| 只召回不精排 | 召回结果的排序不够准确，影响最终质量 |
| 只精排不召回 | 没有候选集，无从精排 |
| 先融合再召回 | 逻辑不通，必须先有召回结果才能融合 |
| 跳过查询改写 | 用户原始输入可能不适合检索，影响召回质量 |

**正确顺序的逻辑**：

```
查询改写：把用户意图翻译成检索友好的形式
    ↓
多路召回：用多种低成本方法快速筛出候选集
    ↓
融合去重：把多路结果合并为统一列表
    ↓
Rerank精排：用高成本方法精细排序
    ↓
LLM生成：基于最相关的文档生成回答
```

每一层的输出是下一层的输入，形成流水线。每一层解决特定的问题，分工明确。

---

## 四、核心洞察总结

### 4.1 混合检索有效的根本原因

**不同方法的"失败模式"不同**。

向量检索会漏掉精确匹配场景的文档，BM25 会漏掉同义词表达的文档。两种方法的失败案例集合基本不重叠。因此，取并集可以相互补救，大大提高召回率。

用数学语言说：如果方法 A 的召回率是 85%，方法 B 的召回率是 80%，且两者的漏选相互独立，那么 A∪B 的理论召回率是 1 - (1-0.85)×(1-0.80) = 97%。

实际中由于两者的漏选不完全独立，达不到 97%，但通常也能到 90-95%。

### 4.2 分层处理有效的根本原因

**在效率和质量之间做渐进式权衡**。

```
处理数据量：  大 ──────────────────────────────────→ 小
计算复杂度：  低 ──────────────────────────────────→ 高
              
              ┌────────┬────────┬────────┬────────┐
              │  召回  │  融合  │ Rerank │  LLM   │
              └────────┴────────┴────────┴────────┘
              
处理文档数：  100万 →   100个  →  100个  →  5个   →  5个
每文档成本：  极低       低        中       高       极高
```

每一层处理的数据量逐层减少，但每个数据点的处理精度逐层提高。这种"漏斗"结构既保证了总体计算成本可控，又保证了最终结果的质量。

### 4.3 一句话总结

**混合检索的本质是用互补的方法覆盖彼此的盲区，分层处理的本质是用漏斗思维在效率和质量之间取得平衡。**

---

## 五、延伸思考

### 5.1 这套方案能完美解决问题吗？

不能。这套方案只是在当前技术条件下的最优实践，仍然存在局限：

1. **Embedding 模型的根本缺陷没有解决**：如果模型把语义理解错了，后面的环节无法纠正
2. **多层处理增加了复杂度和延迟**：每增加一层都有额外的计算成本和出错可能
3. **需要大量调参**：每层的 Top K、融合权重、Reranker 选择等都需要针对具体场景调优
4. **对长尾查询效果有限**：如果查询涉及知识库中没有的知识，再好的检索也无济于事

### 5.2 未来的演进方向

1. **更好的 Embedding 模型**：能更准确地处理否定词、专业术语、长尾知识
2. **端到端优化**：Embedding 模型、检索策略、Reranker 联合训练，而不是各自独立
3. **自适应检索**：根据查询特点自动选择最合适的检索策略，而不是固定的多路召回
4. **融合数据库**：向量、全文、图、关系型数据在一个数据库中统一处理，减少架构复杂度

### 5. 3 实践建议

对于刚开始构建 RAG 系统的开发者：

1. **先跑通最简单的版本**：单路向量召回 + LLM 生成，建立基线
2. **根据错误案例针对性优化**：发现精确匹配差就加 BM25，发现排序不准就加 Reranker
3. **不要过早优化**：在没有足够多真实查询验证之前，复杂的架构可能是过度设计
4. **持续监控和迭代**：没有一劳永逸的方案，需要根据实际效果不断调整